In [ ]:
# import libraries
import os
import requests

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn import linear_model
from sklearn import model_selection
from sklearn import metrics
from sklearn import neighbors
from sklearn import pipeline
from sklearn import preprocessing
from sklearn import tree

# set random_state
random_state = 42

# a)
50  points)  After  exploring  the  data,  build  numeric  prediction  models  that  predict  Spending.  Use linear regression, k-NN, and regression tree techniques. Briefly discuss the models you have built. Use cross-validation with 10 folds to estimate the generalization performance. Present the results for each of the three techniques and discuss which one yields the best performance. 

part a is worth 50 points in total:
* 15 points for exploring the data (i.e., descriptive statistics including min max mean and stdv, visualizations, target 
variable distribution)
* 10 points for correctly building linear regression model - provide screenshots and explain what you are doing and 
the corresponding results 
* 10  points  for  correctly  building  k-NN  model  -  provide  screenshots  and  explain  what  you  are  doing  and  the 
corresponding results 
* 10 points for correctly building regression tree model - provide screenshots and explain what you are doing and the 
corresponding results 
* 5 points for discussing which of the three models yields the best performance

## Data Exploration

In [ ]:
# create file path
path = os.getcwd()
f = 'HW4.xlsx'
f_path = os.path.join(path, f)

# import data
data = pd.read_excel(f_path)

# set index
data = data.set_index(data.sequence_number)
data.drop('sequence_number', axis=1, inplace=True)

# rename columns for easy of use
col_dict = {'Web order': 'web_order', 'Gender=male': 'gender'}
data.rename(col_dict, axis=1, inplace=True)

FileNotFoundError: ignored

In [ ]:
# data head
data.head()

In [ ]:
# data info
data.info()

### Categorical Variables

In [ ]:
def make_bar_plot(col, data=data):
  plt.rcParams["figure.figsize"] = (20,3)
  data[col].value_counts().plot(kind='barh', xlabel = 'value', ylabel='count', title = '{} count'.format(col))
  plt.xlim([0, 2000])
  plt.show()

In [ ]:
make_bar_plot('US')

In [ ]:
make_bar_plot('web_order')

In [ ]:
make_bar_plot('gender')

In [ ]:
make_bar_plot('Address_is_res')

In [ ]:
make_bar_plot('Purchase')

**Categorical Insights:**
* mainly US
* mainly commercial
* ~even split by web_order
* ~even split by gender
* ~even split by purchase

### Quantitative Variables

In [ ]:
# quantitative columns
quant_cols = [
    'Freq',
    'last_update_days_ago',
    '1st_update_days_ago'
]

In [ ]:
# describe the quantitative columns
data[quant_cols].describe()

In [ ]:
# make a bar plot for every quant column
for col in quant_cols:
  data[col].plot(kind='box', vert=False, title = '{} distribution'.format(col))
  plt.show()

In [ ]:
# correlation between quant columns
corr = data[quant_cols].corr()
corr.style.background_gradient(cmap='gray')

### Target Variable

In [ ]:
# box plot for target variable
data['Spending'].plot(kind='box', vert=False, title = '{} distribution'.format('Spending'))
plt.show()

In [ ]:
# histogram with bins=30 to show distribution
plt.hist(data['Spending'], edgecolor="black", bins=40)
plt.title("Spending Distribution")
plt.xlabel("Spending")
plt.ylabel("Count")
plt.show()

## Linear Regression

In [ ]:
# split into X and y
target = 'Spending'
X = data.drop(target, axis=1).copy()
y = data[target].copy()

# set our scoring metric
scoring = 'neg_root_mean_squared_error'

In [ ]:
# build a basic linear model
clf_lm = linear_model.LinearRegression(n_jobs=-1)

# 10 fold CV to assess generalization performance
scores = model_selection.cross_val_score(clf_lm,
    X,
    y,
    scoring=scoring,
    cv=10
)

print("{} performance: {} (+/- {})".format(scoring, round(scores.mean(), 2), round(scores.std(), 2)))

## k-NN

In [ ]:
# scale the data
scaler = preprocessing.StandardScaler()
scaler.fit(X)
X_std = scaler.transform(X)

In [ ]:
# build a bassic kNN regressor with k=5
clf_knn = neighbors.KNeighborsRegressor(
    n_neighbors=5,
    p=2,                     
    metric='minkowski',
    n_jobs=-1)

# 10 fold CV to assess generalization performance
# use the scaled data
scores = model_selection.cross_val_score(clf_knn,
    X_std,
    y,
    scoring=scoring,
    cv=10
)

print("{} performance: {} (+/- {})".format(scoring, round(scores.mean(), 2), round(scores.std(), 2)))

## Regression Tree

In [ ]:
# build a basic regression tree
clf_tree = tree.DecisionTreeRegressor(criterion='squared_error', 
                                      max_depth=3,
                                      random_state=random_state)

# 10 fold CV to assess generalization performance
scores = model_selection.cross_val_score(clf_tree,
    X,
    y,
    scoring=scoring,
    cv=10
)

print("{} performance: {} (+/- {})".format(scoring, round(scores.mean(), 2), round(scores.std(), 2)))

## Modeling Discussion
We chose we to use **negative_root_mean_square_error** as our scoring metric, because it makes sense to penalize larger errors in this context. The negative is required by sklearn; we interpret errors closer to zero as being better.

Given our scoring metric and 10 fold cv, we determined that the **Logistic Regression** model has the best initial generalization performance both in terms of bias (-119.96) and variance (25.89).

# b)
50  points)  Engage  in  feature  engineering  (i.e.,  create  new  features  based  on  existing  features)  to optimize  the  performance  of  linear  regression,  k-NN,  and  regression  tree  techniques.  Present  the 
results for each of the three techniques (choose the best performing model for each technique in case you  try  multiple  models)  and  discuss  which  of  the  three  yields  the  best  performance.  Use  cross-validation  with  10  folds  to  estimate  the  generalization  performance.  Discuss  whether  and  why  the generalization performance was improved or not.

part b is worth 50 points in total:
* 10 points for correctly building the new linear regression model and improving the performance as much as possible 
  * provide screenshots and explain what you are doing and the corresponding results 
* 10 points for correctly building the new k-NN model and improving the performance as much as possible - provide 
screenshots and explain what you are doing and the corresponding results 
* 10 points for correctly building the new regression tree model and improving the performance as much as possible - 
provide screenshots and explain what you are doing and the corresponding results 
* 20 points for discussing if the generalization performance was improved or not for each of the techniques (linear 
regression, kNN, and regression tree) and justifying why it was improved or alternatively why it was not improved

## Feature Engineering

In [ ]:
plt.hist(data['Freq'], edgecolor="red", bins=30)
plt.title('Freq Distribution')
plt.xlabel('frequency')
plt.ylabel('cnt')
plt.show()

In [ ]:
# manual feature engineering
X_tf = X.copy()

# frequent shopper
X_tf['frequent_shopper'] = X_tf['Freq'].apply(lambda x: 1 if x >= 3 else 0)

# days b/w first and last update
X_tf['days_between_update'] = X_tf['1st_update_days_ago'] - X_tf['last_update_days_ago'] 

# female shopping and shipping to residential address
X_tf['female_shopping_residential'] = (1-X_tf['gender']) + X_tf['Address_is_res']

# we think Freq is a) skewed and b) can be modelled as Poisson process so we take the
X_tf['sq_rt_Freq'] = X_tf['Freq']**(1/2)

# all three together
X_tf['all_three'] = X_tf['Freq'] * X_tf['last_update_days_ago'] * X_tf['1st_update_days_ago']

In [ ]:
# automated feature engineering
for col in quant_cols:
  col2 = col + '2'
  col3 = col + '3'

  X_tf[col2] = X_tf[col]**2
  X_tf[col3] = X_tf[col]**3

**Note:** we experimented with the sklearn.preprocessing.polynomial() transformation but saw worse -RMSE performance than the approach outlined above.

## Improved Linear Regression

In [ ]:
clf_lm = linear_model.LinearRegression(n_jobs=-1)

scores = model_selection.cross_val_score(clf_lm,
    X_tf, 
    y,
    scoring=scoring,
    cv=10
)

print("{} performance: {} (+/- {})".format(scoring, round(scores.mean(), 2), round(scores.std(), 2)))

## Improved k-NN

In [ ]:
# scale the data
scaler = preprocessing.StandardScaler()
scaler.fit(X_tf)
X_tf_std = scaler.transform(X_tf)

In [ ]:
clf_knn = neighbors.KNeighborsRegressor(
    n_neighbors=5,
    p=2,                     
    metric='minkowski',
    n_jobs=-1)

# use the scaled version
scores = model_selection.cross_val_score(clf_knn,
    X_tf_std,
    y,
    scoring=scoring,
    cv=10
)

print("{} performance: {} (+/- {})".format(scoring, round(scores.mean(), 2), round(scores.std(), 2)))

## Improved Regression Tree

In [ ]:
clf_tree = tree.DecisionTreeRegressor(criterion='squared_error', 
                                      max_depth=3,
                                      random_state=random_state)

scores = model_selection.cross_val_score(clf_tree,
    X_tf,
    y,
    scoring=scoring,
    cv=10
)

print("{} performance: {} (+/- {})".format(scoring, round(scores.mean(), 2), round(scores.std(), 2)))

## Improved Modeling Discussion
Here we chose to do two kinds of feature engineering.

First, we did manual feature engineering where we tried to inject domain expertise into our model. Our features are as follows:

* `frequent_shopper`: codify people who shop >= 3 as "frequent shoppers" who may or may not buy more per transaction
* `days_between_update`: days between 1st and last update
* `female_shopping_residential`: female shoppers sending packages to residential addresses
* `sqrt_Freq`: square root of Freq to make more linear
* `all_three`: interaction term between Freq, last_update_days_ago,1st_update_days_ago 

Next, we did semi-automated feature engineering on the quantitative columns by raising each to the second and third power.

**Note**: we did **not** incorporate the `preprocessing.Polynomial` approach that was demonstrated in class as we did not notice a better fit and we found the results to be less interpretable.

---

Given our scoring metric, 10 fold cv, and this feature engineering, our  **Logistic Regression** still has the best generalization performance in terms of bias (-113.85) but not necessarily the lowest variance (27.97). The bias is better than in part A, but our variance for Logistic Regression has increased. 

The kNN model improved in terms of bias (-126.85) and variance (25.13).

The regression tree model got worse in terms of bias (-126.38) but the variance is relatively similar (26.45).

The impact to logistic regression makes since as we have given it more parameters to fit on (which will decrease bias and increase variance). 

The kNN's overall fit got better, which is likely the result of our features improving its ability to determine similarity using distance. 

Finally, our regression tree got worse in terms of bias. We suspect this has to do with the manner in which splitting occurs. Decision trees are a "greedy" algorithm and therefore unstable in terms of splitting behavior (sometimes resulting in a different/worse tree).

# c)
Engage in parameter tuning to optimize the performance of linear regression, k-NN, and regression tree techniques. Use cross-validations with 10 folds to estimate the generalization performance. Present the results for each of the three techniques and discuss which one yields the best 
performance.

part a is worth 35 points in total:
* 10 points for correctly optimizing at least two parameters for linear regression model and improving the performance 
as much as possible - provide screenshots and explain what you are doing and the corresponding results 
* 10 points for correctly optimizing at least two parameters for linear k-NN model and improving the performance as 
much as possible - provide screenshots and explain what you are doing and the corresponding results
* 10  points  for  correctly  optimizing  at  least  two  parameters  for  linear  regression  tree  model  and  improving  the 
performance as much as possible - provide screenshots and explain what you are doing and the corresponding results
* 5 points for discussing which of the three models yields the best performance

In [ ]:
# folds for nested cross-validation
inner_cv = model_selection.KFold(n_splits=10, shuffle=True, random_state=random_state)

outer_cv = model_selection.KFold(n_splits=10, shuffle=True, random_state=random_state) # outer is set to 10

## Tuned Linear Regression

In [ ]:
clf_lr = linear_model.LinearRegression(n_jobs=-1)

# two parameters to optimize
fit_intercept_list = [True, False]
positive_list = [True, False]

# select the optimal parameters
gs_lr = model_selection.GridSearchCV(
    estimator=clf_lr,
    param_grid=[{
        'fit_intercept': fit_intercept_list,
        'positive': positive_list}],
    scoring=scoring,                                      
    cv=inner_cv) 

gs_lr_fit = gs_lr.fit(X_tf,y)

print("Non-nested Tuning:")
print("  Model: ", gs_lr_fit.best_estimator_)
print("  Parameterization: ", gs_lr_fit.best_params_)
print("  Non-nested CV {} score: ".format(scoring), round(gs_lr_fit.best_score_, 2))


# generalization performance
nested_gs_lr_fit= model_selection.cross_val_score(
    gs_lr_fit, 
    X=X_tf, 
    y=y, 
    scoring=scoring,
    cv=outer_cv)

print("\nNested Tuning:")
print("  Nested CV {} mean: ".format(scoring), round(nested_gs_lr_fit.mean(), 2), " with st.dev (+/-): ", round(nested_gs_lr_fit.std(),2))

## Tuned kNN

In [ ]:
clf_knn = neighbors.KNeighborsRegressor(
    p=2,                     
    metric='minkowski',
    n_jobs=-1)

# two parameters to optimize
weights_list = ['uniform', 'distance']
n_neighbors_list = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]

# select the optimal parameters
gs_knn = model_selection.GridSearchCV(estimator=clf_knn,
                                      param_grid=[{
                                          'weights': weights_list,
                                          'n_neighbors': n_neighbors_list}],
                                      scoring=scoring,                                      
                                      cv=inner_cv) 


gs_knn_fit = gs_knn.fit(X_tf_std,y)

print("Non-nested Tuning:")
print("  Model: ", gs_knn_fit.best_estimator_)
print("  Parameterization: ", gs_knn_fit.best_params_)
print("  Non-nested CV {} score: ".format(scoring), round(gs_knn_fit.best_score_, 2))

nested_gs_knn_fit= model_selection.cross_val_score(gs_knn_fit, 
                                                   X=X_tf_std, 
                                                   y=y, 
                                                   scoring=scoring,
                                                   cv=outer_cv)


# generalization performance
print("\nNested Tuning:")
print("  Nested CV {} mean: ".format(scoring), round(nested_gs_knn_fit.mean(), 2), " with st.dev (+/-): ", round(nested_gs_knn_fit.std(),2))

## Tuned Regression Tree 

In [ ]:
# create a model
clf_tree = tree.DecisionTreeRegressor(criterion='squared_error',
                                      random_state=random_state)

# two parameters to optimize
min_samples_split_list = [2, 4, 6, 8, 10]
max_depth_list = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, None]

# select the optimal parameters 
gs_tree = model_selection.GridSearchCV(estimator=clf_tree,
                                       param_grid=[{
                                           'min_samples_split': min_samples_split_list,
                                           'max_depth': max_depth_list}],
                                       scoring=scoring,                                      
                                       cv=inner_cv) 

gs_tree_fit = gs_tree.fit(X_tf,y)

print("Non-nested Tuning:")
print("  Model: ", gs_tree_fit.best_estimator_)
print("  Parameterization: ", gs_tree_fit.best_params_)
print("  Non-nested CV {} score: ".format(scoring), round(gs_tree_fit.best_score_, 2))

# generalization performance
nested_gs_tree_fit = model_selection.cross_val_score(gs_tree_fit, 
                                                     X=X_tf, 
                                                     y=y, 
                                                     scoring=scoring,
                                                     cv=outer_cv)

print("nested Tuning:")
print("  Nested CV {} mean: ".format(scoring), round(nested_gs_tree_fit.mean(),2), " with st.dev (+/-): ", round(nested_gs_tree_fit.std(),2))

## Tuned Modeling Discussion
Given our scoring metric, 10 fold cv, feature engineering, and parameter tuning, our **Logistic Regression** once again has the best generalization performance in terms of bias (-115.78) but once again has the lowest variance (20.91). 

Both of these values are better than the Logistic Regression from part A.

Our bias is slightly worse in part C versus the Logistic Regression from part B, but our bias is better. This may ultimately be a better model as the prediction results would be more consistent while still remaining accurate.